## Medallion Architecture

### Bronze Layer

In [1]:
import pandas as pd
import sqlite3

In [2]:
def read_csv_to_dfs(file_paths, df_names, globals_dict):

    if len(file_paths) != len(df_names):
        raise ValueError("The number of file paths and DataFrame names must be the same.")
    
    for file_path, name in zip(file_paths, df_names):
        globals_dict[name] = pd.read_csv(file_path)

In [3]:
file_paths = ['project_1/911_Calls/LAPD_Calls_for_Service_2021.csv','project_1/911_Calls/LAPD_Calls_for_Service_2022.csv','project_1/911_Calls/LAPD_Calls_for_Service_2023.csv']
df_names = ['call_911_21','call_911_22','call_911_23']

read_csv_to_dfs(file_paths, df_names, globals())

In [4]:
call_911_21.head()

,Incident_Number,Area_Occ,Rpt_Dist,Dispatch_Date,Dispatch_Time,Call_Type_Code,Call_Type_Text
0,PD21061400005156,Outside,NaN,06/14/2021 12:00:00 AM,23:28:27,902,TRAFFIC STOP
1,PD21061100000067,Northeast,1116.0,06/11/2021 12:00:00 AM,00:22:10,507P,PARTY
2,PD21061400005022,Outside,NaN,06/14/2021 12:00:00 AM,22:53:07,006,CODE 6
3,PD21061200000206,Outside,NaN,06/12/2021 12:00:00 AM,00:41:01,006,CODE 6
4,PD21061300004840,Hollenbeck,427.0,06/13/2021 12:00:00 AM,22:12:19,594H,CZN HLDG


In [7]:

def save_dfs_to_sqlite(dataframes, table_names, db_name):
    
    if len(dataframes) != len(table_names):
        raise ValueError("The number of DataFrames and table names must be the same.")

    conn = sqlite3.connect(db_name)
    
    for df, table_name in zip(dataframes, table_names):
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Table '{table_name}' created successfully in {db_name}.")
    
    conn.close()


In [8]:
dataframes = [call_911_21,call_911_21,call_911_21] 
table_names = ["call_911_21", "call_911_22", "call_911_23"]

save_dfs_to_sqlite(dataframes, table_names, 'bronze_layer.db')


Table 'call_911_21' created successfully in bronze_layer.db.
Table 'call_911_22' created successfully in bronze_layer.db.
Table 'call_911_23' created successfully in bronze_layer.db.


### SQL QUERIES

In [55]:
conn_bronze = sqlite3.connect('bronze_layer.db')
cursor = conn_bronze.cursor()
cursor.execute('''SELECT COUNT(*) FROM "call_911_21";''')
count_result = cursor.fetchone()[0]  # Fetch the result and get the count
print(f"Number of rows in 911_transformed: {count_result}")

cursor.execute('''SELECT * FROM call_911_21 LIMIT 10 ;''')
rows = cursor.fetchall()

if rows:
    for row in rows:
        print(row)
else:
    print("No data found.")
conn_bronze.close()

Number of rows in 911_transformed: 1579265
('PD21061400005156', 'Outside', None, '06/14/2021 12:00:00 AM', '23:28:27', '902', 'TRAFFIC STOP')
('PD21061100000067', 'Northeast', 1116.0, '06/11/2021 12:00:00 AM', '00:22:10', '507P', 'PARTY')
('PD21061400005022', 'Outside', None, '06/14/2021 12:00:00 AM', '22:53:07', '006', 'CODE 6')
('PD21061200000206', 'Outside', None, '06/12/2021 12:00:00 AM', '00:41:01', '006', 'CODE 6')
('PD21061300004840', 'Hollenbeck', 427.0, '06/13/2021 12:00:00 AM', '22:12:19', '594H', 'CZN HLDG')
('PD21061200003555', '77th Street', 1267.0, '06/12/2021 12:00:00 AM', '18:39:20', '620FR', 'FAMILY R/O')
('PD21061200005040', 'Outside', None, '06/12/2021 12:00:00 AM', '22:43:08', '006', 'CODE 6')
('PD21061400000703', 'Outside', None, '06/14/2021 12:00:00 AM', '05:03:01', '006', 'CODE 6')
('PD21061400000478', '77th Street', 1215.0, '06/14/2021 12:00:00 AM', '02:51:54', '503T3J', 'ATT GTA J/O')
('PD21061400002440', 'N Hollywood', 1548.0, '06/14/2021 12:00:00 AM', '13:06:

### Silver Layer

### Combining Data Sources

In [21]:
conn_bronze = sqlite3.connect('bronze_layer.db')
cursor_bronze = conn_bronze.cursor()
cursor_bronze.execute('''
    SELECT * FROM call_911_21
    UNION ALL
    SELECT * FROM call_911_22
    UNION ALL
    SELECT * FROM call_911_23;
''')
combined_data = cursor_bronze.fetchall() 
column_names = [description[0] for description in cursor_bronze.description]

conn_bronze.close()

conn_silver = sqlite3.connect('silver_layer.db')
cursor_silver = conn_silver.cursor()

cursor_silver.execute(f'''
    CREATE TABLE IF NOT EXISTS call_911_combined ({", ".join(column_names)});
''')

cursor_silver.executemany(f'''
    INSERT INTO call_911_combined VALUES ({", ".join(['?'] * len(column_names))});
''', combined_data)

conn_silver.commit()

conn_silver.close()


### TRANSFORMATIONS

In [31]:
conn_silver = sqlite3.connect('silver_layer.db')
df_combined = pd.read_sql_query('SELECT * FROM call_911_combined', conn_silver)

df_transformed = df_combined.drop(['Rpt_Dist'], axis=1)
def add_time_components(df, date_column):
    df['day_of_week'] = pd.to_datetime(df[date_column], format='%m/%d/%Y %I:%M:%S %p').dt.day_name()
    df['month'] = pd.to_datetime(df[date_column], format='%m/%d/%Y %I:%M:%S %p').dt.month_name()
    df['year'] = pd.to_datetime(df[date_column], format='%m/%d/%Y %I:%M:%S %p').dt.year
    df['date'] = pd.to_datetime(df[date_column], format='%m/%d/%Y %I:%M:%S %p').dt.date
    return df

df_transformed = add_time_components(df_transformed,'Dispatch_Date')
df_transformed.to_sql('911_transformed', conn_silver, if_exists='replace', index=False)

conn_silver.close()

### SQL QUERIES

In [66]:
conn_silver = sqlite3.connect('silver_layer.db')
cursor_silver = conn_silver.cursor()

# Execute the COUNT query
cursor_silver.execute('''SELECT COUNT(*) FROM "911_transformed";''')
count_result = cursor_silver.fetchone()[0]  # Fetch the result and get the count

# Print the result
print(f"Number of rows in 911_transformed: {count_result}")

cursor_silver.execute('''SELECT * FROM "911_transformed" LIMIT 10 ;''')
rows = cursor_silver.fetchall()

if rows:
    for row in rows:
        print(row)
else:
    print("No data found.")
conn_silver.close()

Number of rows in 911_transformed: 4737795
('PD21061400005156', 'Outside', '06/14/2021 12:00:00 AM', '23:28:27', '902', 'TRAFFIC STOP', 'Monday', 'June', 2021, '2021-06-14')
('PD21061100000067', 'Northeast', '06/11/2021 12:00:00 AM', '00:22:10', '507P', 'PARTY', 'Friday', 'June', 2021, '2021-06-11')
('PD21061400005022', 'Outside', '06/14/2021 12:00:00 AM', '22:53:07', '006', 'CODE 6', 'Monday', 'June', 2021, '2021-06-14')
('PD21061200000206', 'Outside', '06/12/2021 12:00:00 AM', '00:41:01', '006', 'CODE 6', 'Saturday', 'June', 2021, '2021-06-12')
('PD21061300004840', 'Hollenbeck', '06/13/2021 12:00:00 AM', '22:12:19', '594H', 'CZN HLDG', 'Sunday', 'June', 2021, '2021-06-13')
('PD21061200003555', '77th Street', '06/12/2021 12:00:00 AM', '18:39:20', '620FR', 'FAMILY R/O', 'Saturday', 'June', 2021, '2021-06-12')
('PD21061200005040', 'Outside', '06/12/2021 12:00:00 AM', '22:43:08', '006', 'CODE 6', 'Saturday', 'June', 2021, '2021-06-12')
('PD21061400000703', 'Outside', '06/14/2021 12:00:00

### Gold Layer

In [90]:

conn_silver = sqlite3.connect('silver_layer.db')
df = pd.read_sql_query('SELECT * FROM "911_transformed"', conn_silver)


# Count of incidents by Location
df_location_count = df.groupby('Area_Occ').size().reset_index(name='incident_count')


# Count of incidents per day of the week
df_day_of_week_count = df.groupby('day_of_week').size().reset_index(name='incident_count')


conn_gold = sqlite3.connect('gold_layer.db')


df_location_count.to_sql('location_incident_summary', conn_gold, if_exists='replace', index=False)
df_day_of_week_count.to_sql('day_of_week_summary', conn_gold, if_exists='replace', index=False)

conn_silver.close()
conn_gold.close()


In [92]:

conn_gold = sqlite3.connect('gold_layer.db')
cursor = conn_gold.cursor()


cursor.execute('''SELECT * FROM location_incident_summary;''')
location_incidents = cursor.fetchall()
print("Location Incident Summary:")
for row in location_incidents:
    print(row)



cursor.execute('''SELECT * FROM day_of_week_summary;''')
day_of_week_summary = cursor.fetchall()
print("\nDay of Week Summary:")
for row in day_of_week_summary:
    print(row)

# Close the connection
conn_gold.close()


Location Incident Summary:
('77th Street', 192270)
('Central', 174201)
('Devonshire', 118011)
('Foothill', 115431)
('Harbor', 120882)
('Hollenbeck', 125127)
('Hollywood', 168582)
('Mission', 141438)
('N Hollywood', 149109)
('Newton', 159840)
('Northeast', 127254)
('Olympic', 146472)
('Outside', 1734408)
('Pacific', 163461)
('Rampart', 141279)
('Southeast', 141165)
('Southwest', 149982)
('Topanga', 121596)
('Van Nuys', 129996)
('West LA', 142203)
('West Valley', 124374)
('Wilshire', 150714)

Day of Week Summary:
('Friday', 690504)
('Monday', 658698)
('Saturday', 661749)
('Sunday', 663063)
('Thursday', 683889)
('Tuesday', 682437)
('Wednesday', 697455)
